In [1]:
import numpy as np
from datetime import datetime
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from glob import glob
import pickle
from astropy.time import Time
import matplotlib.ticker as mticker

/home/apizzuto/.venvs/py3_realtime/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


# Load in the nova dataframe, format to a latex table

In [42]:
novae = pd.read_pickle('/home/apizzuto/Nova/master_nova_dataframe.pkl')
np.count_nonzero(novae['gamma'] == True)

16

In [25]:
def clean_date(date):
    return date.iso.split(' ')[0]

def clean_ra(ra):
    return f"{ra:.2f}" + "$^{\circ}$"

def clean_dec(dec):
    return f"{dec:+.2f}" + "$^{\circ}$"

def clean_gam(gamma):
    return '\checkmark' if gamma else '--'

In [37]:
novae = pd.read_pickle('/home/apizzuto/Nova/master_nova_dataframe.pkl')
novae = novae.sort_values('Date')
novae = novae[["Name", "Date", "RA", "Dec", "Peak", "gamma", "refs"]]
for col, cleaner in [('Date', clean_date), ('RA', clean_ra), ('Dec', clean_dec),
                    ('gamma', clean_gam)]:
    novae[col] = novae[col].apply(cleaner)
    
novae = novae.rename(columns={'Date': 'Peak Date', 'RA': '$\\alpha$',
                             'Dec': '$\delta$', 'gamma': '$\gamma$-detected',
                             'refs': 'Reference', 'Peak': "Peak Mag."})
    
latex_str = novae.to_latex(longtable=True, index_names=False, index=False)
latex_str = latex_str.replace('\\textasciicircum ', '^')
latex_str = latex_str.replace('\$', '$')
latex_str = latex_str.replace('\\textbackslash ', '\\')
latex_str = latex_str.replace('\\{', '{')
latex_str = latex_str.replace('\\}', '}')
with open('./tmp_latex_table_paper.tex', 'w') as fi:
    fi.writelines(latex_str)

In [3]:
formatter = mticker.ScalarFormatter(useOffset=False, useMathText=True)
sci_not = lambda x, dec: "${}$".format(formatter._formatSciNotation('%1.{}e'.format(dec) % x))

In [4]:
external_dict = {'Name': [], 'R.A.': [], 'dec.': [], 'Start Time': [], 'Duration': [],
                'ns': [], 'logp': [], 'Upper Limit': [], 'Energy Range': [], 'Reference': [], 
                'Issued Alert': []}
txs_counter = 0
for f in external_results_fs:
    analysis = glob(f + '/*results.pickle')
    with open(analysis[0], 'r') as pkl:
        res = pickle.load(pkl)
    external_dict['Name'].append(res['name'])
    external_dict['R.A.'].append('{:.2f}'.format(np.degrees(res['ra'])))
    external_dict['dec.'].append('{:+.2f}'.format(np.degrees(res['dec'])))
    start_mjd = res['start']
    external_dict['Start Time'].append(Time(start_mjd, format='mjd').iso)
    stop_mjd = res['stop']
    duration = (stop_mjd - start_mjd)*86400.
    external_dict['Duration'].append(sci_not(duration, 2))
    external_dict['ns'].append('{:.2f}'.format(res['ns']))
    logp = -1.*np.log10(res['p'])
    logp = logp if logp != -0.0 else 0.0
    external_dict['logp'].append('{:.2f}'.format(logp))
    up_lim = res['upper_limit']*duration*1e6
    external_dict['Upper Limit'].append(sci_not(up_lim, 1))
    external_dict['Energy Range'].append('({},{})'.format(sci_not(res['low_en'], 0), sci_not(res['high_en'], 0)))
    external_dict['Reference'].append('|')
    external_dict['Issued Alert'].append('|')
#     try:
#         grbllh_res.loc[res['name']]
#     except:
#         if res['name'] == 'TXS 0506+056':
#             map_name = res['name'] + '_{}'.format(txs_counter)
#             txs_counter += 1
#         else:
#             map_name = res['name']
#         grbllh_name = misname_map[map_name]
#         ind = grbllh_res.loc(grbllh_name)
#         external_dict['Reference'].append(grbllh_res[''])
#         external_dict['Issued Alert'].append(res['name'])

In [5]:
external_df = pd.DataFrame.from_dict(external_dict)
external_df = external_df[[u'Name', u'R.A.', u'dec.', u'Start Time', u'Duration', u'ns',
                         u'logp', u'Upper Limit', u'Energy Range', 'Reference', 'Issued Alert']]
external_df = external_df.sort_values('Start Time')
#external_df = external_df.set_index('Name')

In [6]:
latex_str = external_df.to_latex(longtable=True, index_names=False, index=False)
latex_str = latex_str.replace('\\textasciicircum ', '^')
latex_str = latex_str.replace('\\$', '$')
latex_str = latex_str.replace('mes\\}10', 'mes 10')
latex_str = latex_str.replace('\\{\\textbackslash ', '\\')
latex_str = latex_str.replace('\\{', '{')
latex_str = latex_str.replace('\\}', '}')
#print(latex_str[-100:])
#latex_str = r'{}'.format(latex_str)
#latex_str = latex_str.replace('\\textbackslash ', '\\' )
#latex_esc = latex_str.encode('unicode_escape')
with open('./tmp_latex_table_paper.txt', 'w') as fi:
    fi.writelines(latex_str)

In [7]:
external_df

,Name,R.A.,dec.,Start Time,Duration,ns,logp,Upper Limit,Energy Range,Reference,Issued Alert
1,CygnusX3,308.11,+40.96,2017-04-03 00:00:00.000,$8.64{\times}10^{4}$,0.00,0.00,$5.2{\times}10^{-2}$,"($7{\times}10^{2}$,$4{\times}10^{5}$)",|,|
46,GRB170405A,219.83,-25.24,2017-04-05 18:35:49.000,$1.2{\times}10^{3}$,0.00,0.00,$3.2{\times}10^{-1}$,"($7{\times}10^{4}$,$2{\times}10^{7}$)",|,|
3,AGL J0523+0646,80.86,+6.78,2017-04-15 11:50:00.000,$4.32{\times}10^{5}$,0.00,0.00,$3.9{\times}10^{-2}$,"($10^{3}$,$3{\times}10^{6}$)",|,|
50,AT2017eaw,308.68,+60.19,2017-05-10 12:00:00.000,$2.59{\times}10^{5}$,0.23,0.89,$7.4{\times}10^{-2}$,"($6{\times}10^{2}$,$2{\times}10^{5}$)",|,|
17,FermiJ1544-0649,236.08,-6.82,2017-05-15 00:00:00.000,$2.74{\times}10^{5}$,0.00,0.00,$5.1{\times}10^{-2}$,"($2{\times}10^{3}$,$9{\times}10^{6}$)",|,|
21,FermiJ1544-0649,236.08,-6.82,2017-05-18 04:04:40.000,$9.36{\times}10^{5}$,0.00,0.00,$5.6{\times}10^{-2}$,"($2{\times}10^{3}$,$9{\times}10^{6}$)",|,|
4,AXP 4U 0142+61,26.59,+61.75,2017-07-13 22:54:33.000,$7.2{\times}10^{3}$,0.00,0.00,$5.9{\times}10^{-2}$,"($5{\times}10^{2}$,$2{\times}10^{5}$)",|,|
23,ULGRB RTDE 170714A,34.35,+1.99,2017-07-14 11:25:32.000,$4.36{\times}10^{4}$,0.00,0.00,$3{\times}10^{-2}$,"($10^{3}$,$5{\times}10^{6}$)",|,|
24,AT2017fro,259.98,+41.68,2017-07-22 00:00:00.000,$1.21{\times}10^{6}$,0.00,0.00,$6.1{\times}10^{-2}$,"($7{\times}10^{2}$,$3{\times}10^{5}$)",|,|
14,AGL J1412-0522,213.00,-5.40,2017-08-05 03:00:00.000,$1.73{\times}10^{5}$,0.00,0.00,$4{\times}10^{-2}$,"($2{\times}10^{3}$,$8{\times}10^{6}$)",|,|
